# Feature Engineering Pipeline

In [ ]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 http://ppa.launchpad.net

In [ ]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install hopsworks

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-hi88impf
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-hi88impf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 7.6 MB 5.2 MB/s 
  Created wheel for transformers: filename=transformers-4.26.0.dev0-py3-none-any.whl size=5858215 sha256=7a107637f2a74f4714254e3ca7462d6b4dd77b6686922a5bf49d344698df0f71
  Stored in directory: /tmp/pip-ephem-wheel-cache-fn4q0i51/wheels/42/68/45/c63edff61c292f2dfd4df4ef6522dcbecc603e7af82813c1d7
Successfully built transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
    

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
import hopsworks
project = hopsworks.login()

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated

Paste it here: ··········
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5287


## Load Dataset

Using 🤗 Datasets, downloading and preparing data is extremely simple. 
We can download and prepare the Common Voice splits in just one line of code. 

First, ensure you have accepted the terms of use on the Hugging Face Hub: [mozilla-foundation/common_voice_11_0](https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0). Once you have accepted the terms, you will have full access to the dataset and be able to download the data locally.

Since Hindi is very low-resource, we'll combine the `train` and `validation` 
splits to give approximately 8 hours of training data. We'll use the 4 hours 
of `test` data as our held-out test set:

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "sv-SE", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "sv-SE", split="test", use_auth_token=True)

print(common_voice)

Extracting data files #4:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #2:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #3:   0%|          | 0/1 [00:00<?, ?obj/s]

...





Reading metadata...: 5699it [00:00, 80819.78it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]






Reading metadata...: 1346it [00:00, 59221.57it/s]


Dataset common_voice_11_0 downloaded and prepared to /root/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/sv-SE/11.0.0/f8e47235d9b4e68fa24ed71d63266a02018ccf7194b2a8c9c598a5f3ab304d9f. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 12360
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 5069
    })
})


Most ASR datasets only provide input audio samples (`audio`) and the 
corresponding transcribed text (`sentence`). Common Voice contains additional 
metadata information, such as `accent` and `locale`, which we can disregard for ASR.
Keeping the notebook as general as possible, we only consider the input audio and
transcribed text for fine-tuning, discarding the additional metadata information:

In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 12360
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 5069
    })
})


### Prepare Data

Let's print the first example of the Common Voice dataset to see 
what form the data is in:

In [ ]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/1aaf2e61fb837413b804938857af61aeb372611d01b80b3f40d9edeb7f354244/common_voice_sv-SE_20466896.mp3', 'array': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 'sampling_rate': 48000}, 'sentence': 'Du ser ut att ha gjort det här hela livet.'}


Since 
our input audio is sampled at 48kHz, we need to _downsample_ it to 
16kHz prior to passing it to the Whisper feature extractor, 16kHz being the sampling rate expected by the Whisper model. 

We'll set the audio inputs to the correct sampling rate using dataset's 
[`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column)
method. This operation does not change the audio in-place, 
but rather signals to `datasets` to resample audio samples _on the fly_ the 
first time that they are loaded:

In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

Re-loading the first audio sample in the Common Voice dataset will resample 
it to the desired sampling rate:

In [ ]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/1aaf2e61fb837413b804938857af61aeb372611d01b80b3f40d9edeb7f354244/common_voice_sv-SE_20466896.mp3', 'array': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 'sampling_rate': 16000}, 'sentence': 'Du ser ut att ha gjort det här hela livet.'}


Now we can write a function to prepare our data ready for the model:
1. We load and resample the audio data by calling `batch["audio"]`. As explained above, 🤗 Datasets performs any necessary resampling operations on the fly.
2. We use the feature extractor to compute the log-Mel spectrogram input features from our 1-dimensional audio array.
3. We encode the transcriptions to label ids through the use of the tokenizer.

In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

We can apply the data preparation function to all of our training examples using dataset's `.map` method. The argument `num_proc` specifies how many CPU cores to use. Setting `num_proc` > 1 will enable multiprocessing. If the `.map` method hangs with multiprocessing, set `num_proc=1` and process the dataset sequentially.

In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=1)

  0%|          | 0/12360 [00:00<?, ?ex/s]

  0%|          | 0/5069 [00:00<?, ?ex/s]

In [ ]:
help(common_voice)

Help on DatasetDict in module datasets.dataset_dict object:

class DatasetDict(builtins.dict)
 |  A dictionary (dict of str: datasets.Dataset) with dataset transforms methods (map, filter, etc.)
 |  
 |  Method resolution order:
 |      DatasetDict
 |      builtins.dict
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, k) -> datasets.arrow_dataset.Dataset
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  align_labels_with_mapping(self, label2id: Dict, label_column: str) -> 'DatasetDict'
 |      Align the dataset's label ID and label name mapping to match an input :obj:`label2id` mapping.
 |      This is useful when you want to ensure that a model's predicted labels are aligned with the dataset.
 |      The alignment in done using the lowercase label names.
 |      
 |      Args:
 |          label2id (:obj:`dict`):
 |              The label name to ID mapping to align the dataset with.
 |          label_column 

### Upload data to google drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

## Push to the drive directly
common_voice.save_to_disk("/content/drive/MyDrive/KTH/Fall22/P2/ID2223/lab2/common_voice") ## Set your path

In [ ]:
import os
print(os.listdir("/content/drive/MyDrive/KTH/Fall22/P2/ID2223/lab2/common_voice/"))
print(os.listdir("/content/drive/MyDrive/KTH/Fall22/P2/ID2223/lab2/common_voice/train"))
print(os.listdir("/content/drive/MyDrive/KTH/Fall22/P2/ID2223/lab2/common_voice/test"))

['train', 'test', 'dataset_dict.json']
['state.json', 'dataset_info.json', 'dataset.arrow']
['dataset.arrow', 'state.json', 'dataset_info.json']


In [ ]:
def get_dir_size(path='/content/drive/MyDrive/KTH/Fall22/P2/ID2223/lab2/common_voice/train'):
    total = 0
    with os.scandir(path) as it:
        for entry in it:
            if entry.is_file():
                total += entry.stat().st_size
            elif entry.is_dir():
                total += get_dir_size(entry.path)
    return total
sz = get_dir_size(path="/root/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/")
print(sz)

17419676137


In order to reduce the size, we can compress the common_voice folder and store only the zip.

In [ ]:
!zip -r /content/drive/MyDrive/KTH/Fall22/P2/ID2223/lab2/common_voice.zip -i /content/MyDrive/KTH/Fall22/P2/ID2223/lab2/common_voice/

### Upload Data to Hopsworks

In [ ]:
#upload to hopsworks
dataset_api = project.get_dataset_api()

# create the directories in Hopsworks in the file browser inside some existing dataset
# common_voice/train
# common_voice/test

In [ ]:
# upload all of the local files to the common_voice directory you created in Hopsworks

uploaded_file_path = dataset_api.upload(
    local_path = "/content/drive/MyDrive/KTH/Fall22/P2/ID2223/lab2/common_voice/train/dataset.arrow", 
    upload_path = "ID2223_Lab1_ZS_Training_Datasets/common-voice/train/", overwrite=True)

Uploading: 0.000%|          | 0/11871608720 elapsed<00:00 remaining<?

In [ ]:
uploaded_file_path = dataset_api.upload(
    local_path = "/content/drive/MyDrive/KTH/Fall22/P2/ID2223/lab2/common_voice/train/dataset_info.json", 
    upload_path = "ID2223_Lab1_ZS_Training_Datasets/common-voice/train/", overwrite=True)

Uploading: 0.000%|          | 0/4909 elapsed<00:00 remaining<?

In [ ]:
uploaded_file_path = dataset_api.upload(
    local_path = "/content/drive/MyDrive/KTH/Fall22/P2/ID2223/lab2/common_voice/train/state.json", 
    upload_path = "ID2223_Lab1_ZS_Training_Datasets/common-voice/train/", overwrite=True)

Uploading: 0.000%|          | 0/267 elapsed<00:00 remaining<?

In [ ]:
uploaded_file_path = dataset_api.upload(
    local_path = "/content/drive/MyDrive/KTH/Fall22/P2/ID2223/lab2/common_voice/test/dataset.arrow", 
    upload_path = "ID2223_Lab1_ZS_Training_Datasets/common-voice/test/", overwrite=True)

Uploading: 0.000%|          | 0/4868700344 elapsed<00:00 remaining<?

In [ ]:
uploaded_file_path = dataset_api.upload(
    local_path = "/content/drive/MyDrive/KTH/Fall22/P2/ID2223/lab2/common_voice/test/dataset_info.json", 
    upload_path = "ID2223_Lab1_ZS_Training_Datasets/common-voice/test/", overwrite=True)

Uploading: 0.000%|          | 0/4909 elapsed<00:00 remaining<?

In [ ]:
uploaded_file_path = dataset_api.upload(
    local_path = "/content/drive/MyDrive/KTH/Fall22/P2/ID2223/lab2/common_voice/test/state.json", 
    upload_path = "ID2223_Lab1_ZS_Training_Datasets/common-voice/test/", overwrite=True)

Uploading: 0.000%|          | 0/255 elapsed<00:00 remaining<?